# Install and Import packages

**Machine Learning for Smart Health Systems**</br>
Instructor: Juber Rahman</br>
**Omdena School**</br>
Course Link: https://omdena.com/course/machine-learning-for-smart-health-systems/</br>
Updated: Oct 30, 2021

In [ ]:
# !pip install hrv-analysis

In [ ]:
# !pip install wfdb

In [1]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import posixpath

import wfdb



## Download content from Physionet

In [ ]:
# List the PhysioNet Databases

dbs = wfdb.get_dbs()
display(dbs)

In [2]:
# Download all the WFDB records and annotations from a small PhysioNet Database

# Make a temporary download directory in your current working directory
cwd = os.getcwd()
dl_dir = os.path.join(cwd, 'tmp_dl_dir')

# Download all the WFDB content
# wfdb.dl_database('apnea-ecg', dl_dir=dl_dir)

# Display the downloaded content in the folder
display(os.listdir(dl_dir))

# Cleanup: delete the downloaded directory
# shutil.rmtree(dl_dir)

['x12.apn',
 'c08.dat',
 'x06.apn',
 'x12.hea',
 'x06.hea',
 'a04.hea',
 'a10.hea',
 'a04.apn',
 'a10.apn',
 'x34.dat',
 'c06.hea',
 'x20.dat',
 'x08.dat',
 'c06.apn',
 'x09.dat',
 'c07.apn',
 'x21.dat',
 'c07.hea',
 'x35.dat',
 'a11.apn',
 'a05.apn',
 'c01r.hea',
 'a11.hea',
 'c01r.apn',
 'a05.hea',
 'x07.hea',
 'x13.hea',
 'c09.dat',
 'x07.apn',
 'x13.apn',
 'x05.apn',
 'x11.apn',
 'x05.hea',
 'x11.hea',
 'a13.hea',
 'a07.hea',
 'a13.apn',
 'a07.apn',
 'a09.dat',
 'c02r.dat',
 'x23.dat',
 'c05.hea',
 'b04.qrs',
 'c05.apn',
 'c04.apn',
 'b05.qrs',
 'c10.apn',
 'c04.hea',
 'x22.dat',
 'c10.hea',
 'a20.dat',
 'a08.dat',
 'a06.apn',
 'a12.apn',
 'a06.hea',
 'a12.hea',
 'x10.hea',
 'x04.hea',
 'x10.apn',
 'x04.apn',
 'a03r.dat',
 'a02r.dat',
 'x28.apn',
 'x14.hea',
 'x14.apn',
 'x28.hea',
 'a16.apn',
 'a02.apn',
 'a16.hea',
 'a02.hea',
 'a18.dat',
 'b01.qrs',
 'x26.dat',
 'x32.dat',
 'c01.hea',
 'x33.dat',
 'x27.dat',
 'c01.apn',
 'a19.dat',
 'c03r.dat',
 'a03.hea',
 'a17.hea',
 'a03.apn'

In [3]:
files = os.listdir(dl_dir)
all_records = []

for file in files:
    record = file.split('.',1)[0]
    if record not in all_records and record[-1] != 'r':
        all_records.append(record)

In [4]:
all_records

['x12',
 'c08',
 'x06',
 'a04',
 'a10',
 'x34',
 'c06',
 'x20',
 'x08',
 'x09',
 'c07',
 'x21',
 'x35',
 'a11',
 'a05',
 'x07',
 'x13',
 'c09',
 'x05',
 'x11',
 'a13',
 'a07',
 'a09',
 'x23',
 'c05',
 'b04',
 'c04',
 'b05',
 'c10',
 'x22',
 'a20',
 'a08',
 'a06',
 'a12',
 'x10',
 'x04',
 'x28',
 'x14',
 'a16',
 'a02',
 'a18',
 'b01',
 'x26',
 'x32',
 'c01',
 'x33',
 'x27',
 'a19',
 'a03',
 'a17',
 'x15',
 'x29',
 'x01',
 'x17',
 'x03',
 'a01',
 'a15',
 'c03',
 'x19',
 'b02',
 'x31',
 'x25',
 'x24',
 'c02',
 'x30',
 'b03',
 'x18',
 'a14',
 'x02',
 'x16']

## Pre-processing and peak detection

In [ ]:
!pip install matplotlib==3.1.3

In [5]:
import wfdb
from wfdb import processing
import matplotlib.pyplot as plt
import matplotlib
# # Load the WFDB record 10 sec segment and the physical samples
# record = wfdb.rdrecord('tmp_dl_dir/a01', sampfrom=0, sampto=10000)

# wfdb.plot_wfdb(record=record, time_units='seconds', title='Record a01 from PhysioNet Apnea-ECG') 
# display(record.__dict__)


In [6]:
#Use the GQRS detection algorithm and correct the peaks

def peaks_hr(sig, peak_inds, fs, title, figsize=(20, 10), saveto=None):
    "Plot a signal with its peaks and heart rate"
    # Calculate heart rate
    hrs = processing.hr.compute_hr(sig_len=sig.shape[0], qrs_inds=peak_inds, fs=fs)
    
    N = sig.shape[0]
    
    fig, ax_left = plt.subplots(figsize=figsize)
    ax_right = ax_left.twinx()
    
    ax_left.plot(sig, color='#3979f0', label='Signal')
    ax_left.plot(peak_inds, sig[peak_inds], 'rx', marker='x', 
                 color='#8b0000', label='Peak', markersize=12)
    ax_right.plot(np.arange(N), hrs, label='Heart rate', color='m', linewidth=2)

    ax_left.set_title(title)

    ax_left.set_xlabel('Time (ms)')
    ax_left.set_ylabel('ECG (mV)', color='#3979f0')
    ax_right.set_ylabel('Heart rate (bpm)', color='m')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax_left.tick_params('y', colors='#3979f0')
    ax_right.tick_params('y', colors='m')
    if saveto is not None:
        plt.savefig(saveto, dpi=600)
    plt.show()




In [ ]:
!pip install biosppy

In [7]:
#get r peaks on this segment
import biosppy
# ind= biosppy.signals.ecg.ecg(record.p_signal[:,0], sampling_rate=100, show=True)

In [8]:
# Now load the entire duration of the signal and get r peaks
all_wfdb = []
for record in all_records:
    print(record)
    all_wfdb.append(wfdb.rdrecord('tmp_dl_dir/{}'.format(record), sampfrom=0,sampto=100000))
# ind= biosppy.signals.ecg.ecg(record.p_signal[:,0], sampling_rate=100, show=False)

x12
c08
x06
a04
a10
x34
c06
x20
x08
x09
c07
x21
x35
a11
a05
x07
x13
c09
x05
x11
a13
a07
a09
x23
c05
b04
c04
b05
c10
x22
a20
a08
a06
a12
x10
x04
x28
x14
a16
a02
a18
b01
x26
x32
c01
x33
x27
a19
a03
a17
x15
x29
x01
x17
x03
a01
a15
c03
x19
b02
x31
x25
x24
c02
x30
b03
x18
a14
x02
x16


In [13]:
from hrvanalysis import remove_ectopic_beats, interpolate_nan_values, get_time_domain_features, get_frequency_domain_features, get_geometrical_features
from hrvanalysis import get_csi_cvi_features, get_poincare_plot_features
from hrvanalysis.extract_features import get_frequency_domain_features


all_results = []
for record in all_wfdb:
    print(record.record_name)
    results = {}
    ind= biosppy.signals.ecg.ecg(record.p_signal[:,0], sampling_rate=100, show=False)
    
    # print(ind.keys())

    # print(np.average(ind['heart_rate']))

    rr_ind = np.diff(ind['rpeaks'])

    rr_ms = [element * 10 for element in rr_ind]

    # print(rr_ms)

    # remove ectopic beats and interpolate

    ect_ind = remove_ectopic_beats(rr_ms)
    nn_interval = interpolate_nan_values(rr_intervals=ect_ind)

    time_domain_features = get_time_domain_features(nn_interval)

    ## Time Domain Measures

    time_dict = get_time_domain_features(nn_interval)
    results['avnn'] = time_dict['mean_nni']
    results['hr'] = time_dict['mean_hr']
    results['SDNN'] = time_dict['sdnn']
    results['pNN50'] = time_dict['pnni_20']
    results['RMSSD'] = time_dict['rmssd']
    results['HRdiff'] = time_dict['max_hr'] - time_dict['min_hr']

    ## Frequency Domain Measures
#     Apply a Fast Fourier Transform (FFT) to the time-series data to obtain frequency domain measures.

    freq_dict = get_frequency_domain_features(nn_interval)

    results['VLF'] = freq_dict['vlf']
    results['LF'] = freq_dict['lf']
    results['HF'] = freq_dict['hf']
    results['LHFratio'] = freq_dict['lf_hf_ratio']
    results['Record'] = record.record_name
    
    print(results)

    all_results.append(results)

x12
8 ectopic beat(s) have been deleted with malik rule.
{'avnn': 773.2106893880713, 'hr': 78.21279895529987, 'SDNN': 58.14687826134675, 'pNN50': 29.844961240310077, 'RMSSD': 27.497004769935025, 'HRdiff': 157.00483091787441, 'VLF': 897.0510446157955, 'LF': 592.7677634731058, 'HF': 394.6911157250007, 'LHFratio': 1.5018523089486364, 'Record': 'x12'}
c08
111 ectopic beat(s) have been deleted with malik rule.
{'avnn': 816.0506950122649, 'hr': 81.29720739785986, 'SDNN': 174.1049706339102, 'pNN50': 64.8936170212766, 'RMSSD': 94.32723276574937, 'HRdiff': 817.4077578051088, 'VLF': 3421.633262829001, 'LF': 8270.48422045124, 'HF': 3087.8698969039974, 'LHFratio': 2.6783784604213756, 'Record': 'c08'}
x06
177 ectopic beat(s) have been deleted with malik rule.
{'avnn': 1710.5718270571826, 'hr': 82.84308929140118, 'SDNN': 10387.294674946756, 'pNN50': 74.86033519553072, 'RMSSD': 8147.33931103658, 'HRdiff': 666.3992273976673, 'VLF': 69306186.17466491, 'LF': 776670.921583313, 'HF': 36173.13425659931, 'L

88 ectopic beat(s) have been deleted with malik rule.
{'avnn': 942.6606805293005, 'hr': 68.42101963028117, 'SDNN': 182.3027184610466, 'pNN50': 59.413434247871336, 'RMSSD': 94.556425966615, 'HRdiff': 313.98013750954925, 'VLF': 5436.200989931097, 'LF': 5158.111740139604, 'HF': 3285.783846017807, 'LHFratio': 1.569826860762907, 'Record': 'c05'}
b04
267 ectopic beat(s) have been deleted with malik rule.
{'avnn': 980.754817987152, 'hr': 120.21573766479887, 'SDNN': 3956.7139668802943, 'pNN50': 73.84780278670954, 'RMSSD': 3099.0359027057716, 'HRdiff': 666.0004441483455, 'VLF': 35667475.7183396, 'LF': 2884333.288198973, 'HF': 259833.14438775257, 'LHFratio': 11.10071347901114, 'Record': 'b04'}
c04
889 ectopic beat(s) have been deleted with malik rule.
{'avnn': 415.30524722502525, 'hr': 183.81354755649852, 'SDNN': 324.1939799888238, 'pNN50': 79.70721857647652, 'RMSSD': 158.20364961064013, 'HRdiff': 592.5925925925926, 'VLF': 3795.1550855884157, 'LF': 7361.7575101492885, 'HF': 9588.733052133495, 'L

6 ectopic beat(s) have been deleted with malik rule.
{'avnn': 17441.346153846152, 'hr': 63.09428159813443, 'SDNN': 101765.23660216463, 'pNN50': 57.28155339805825, 'RMSSD': 84112.24163701669, 'HRdiff': 272.6571095979972, 'VLF': 458993671.8487386, 'LF': 70464312.60738635, 'HF': 6001842.762956533, 'LHFratio': 11.74044629131126, 'Record': 'a03'}
a17
0 ectopic beat(s) have been deleted with malik rule.
{'avnn': 761.7786259541984, 'hr': 79.00978961388176, 'SDNN': 42.26725513658649, 'pNN50': 21.543162719633308, 'RMSSD': 20.5983823079658, 'HRdiff': 27.815934065934073, 'VLF': 605.1158060445198, 'LF': 422.98102643299455, 'HF': 170.95090136812922, 'LHFratio': 2.4742836864143722, 'Record': 'a17'}
x15
28 ectopic beat(s) have been deleted with malik rule.
{'avnn': 926.2210915818687, 'hr': 66.2545410505926, 'SDNN': 106.67290849407418, 'pNN50': 38.7037037037037, 'RMSSD': 48.9316416601319, 'HRdiff': 217.3913043478261, 'VLF': 2397.1420002780096, 'LF': 2282.4480127656407, 'HF': 907.5970948681318, 'LHFrat

In [ ]:
from hrvanalysis import plot_psd

plot_psd(nn_interval, method="welch")

In [ ]:
print(results)

In [ ]:
print(len(nn_interval))

## Poincare Plot features

In [ ]:
from hrvanalysis import plot_poincare

plot_poincare(nn_interval)
plot_poincare(nn_interval, plot_sd_features=True)

In [ ]:
# results['record']='a01'

In [ ]:
all_results

In [ ]:
results.keys()

In [14]:
import csv
csv_columns = results.keys()

csv_file = "features.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for results in all_results:
            writer.writerow(results)
        
except IOError:
    print("I/O error")